In [22]:
import requests
import random
import argparse
import itertools

def getChapterTasks(chapters):
    chapters_to_include = chapters
    chapters_json = requests.get('http://uhunt.felix-halim.net/api/cpbook/3').json()
    chapters = [chapters_json[x - 1] for x in chapters_to_include]
    tasks = [sub_sub_chapter for chapter in chapters for sub_chapter in chapter['arr'] for sub_sub_chapter in sub_chapter['arr']]
    tasks = filter(lambda task: type(task) == int, [task for array in tasks for task in array])
    tasks = map(lambda task: -task if task < 0 else task, tasks)
    return tasks

def getUserSubmissions(username):
    user_id = int(requests.get('http://uhunt.felix-halim.net/api/uname2uid/{0}'.format(username)).text)
    pids = set([sub[1] for sub in requests.get('http://uhunt.felix-halim.net/api/subs-user/{0}'.format(user_id)).json()['subs']])
    print username
    print user_id
    return pids
    
def getNextTaskToSolve(tasks, pids):
    random.shuffle(tasks)
    while True:
        taskIndex = random.randint(0, len(tasks) - 1)
        nextPid = requests.get('http://uhunt.felix-halim.net/api/p/num/{0}'.format(tasks[taskIndex])).json()['pid']
        if nextPid in pids:
            del tasks[taskIndex]
        else:
            return tasks[taskIndex]
            
def handleArguments():
    parser = argparse.ArgumentParser(description='Generate the next task to solve on UVa from textbook (3rd edition)')
    parser.add_argument('username', action='store', nargs=1, type=str, help='username of profile to generate next unsolved task')
    parser.add_argument('--cns', action='store', nargs='*', default=range(1,10), 
                        type=int, required=False,
                        help='number of chapters to generate tasks from')
    return parser.parse_args()
    
args = handleArguments()
tasks = getChapterTasks(args.cns)
pids = getUserSubmissions(args.username[0])
print 'Next task to solve: {0}'.format(getNextTaskToSolve(tasks, pids))

usage: __main__.py [-h] [--cns [CNS [CNS ...]]] username
__main__.py: error: unrecognized arguments: -f


SystemExit: 2